In [1]:
import QUANTAXIS as qa
from QIFIAccount import QIFI_Account,QA_Position
from QIFIAccount.QASimAccount import QIFI_SIM_Account
import pandas as pd

提示：当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口 panel 参数将固定为 False
注意：0.25 以上版本 pandas 不支持 panel，如使用该数据结构和相关函数请注意修改


In [9]:
username='myacc18'
my=QIFI_SIM_Account(username=username,password=username)
my.initial()

Create new Account


In [10]:
df=pd.read_excel("E:\\leo\\投资\\2021\\股票持仓分析20210622-快照20210622.xlsx",sheet_name="持仓明细详情",header=2)
df=df[df['pos_num']>0]
code_cn_list=list(x.split(".")[0] for x in df.code[df['currency']=='CNY'].unique().tolist())
code_cn_list
code_hk_list=list(x.split(".")[0] for x in df.code[df['currency']=='HK'].unique().tolist())
code_hk_list=[i.rjust(5,'0') if len(i)==4  else i for i in code_hk_list]

In [11]:
def s(x):
    print(type(x['code']),x['code'].split('.'))
def add_position(stock):
    code=stock['code'].split('.')
    flag=False
#     print(code[0],stock['currency'],stock['code'][0])
    if len(code[0])==6 and stock['currency']=='CNY':
        code=code[0]
        market_type=qa.MARKET_TYPE.STOCK_CN
        exchange_id=qa.EXCHANGE_ID.SZSE if code[0] in ('0','3') else qa.EXCHANGE_ID.SSE
        flag=True
    elif len(code[0])==6:
        code=code[0]
        market_type='stock_cn_b'
        exchange_id=qa.EXCHANGE_ID.SZSE
        flag=False
    else:
        code=str(code[0]).rjust(5,'0')
        market_type='stock_hk'
        exchange_id='hk'
        flag=False
    if flag==True:
        pos=QA_Position(
                code=code,
                account_cookie='myacc17',
                market_type=market_type,
                exchange_id=exchange_id,
                volume_long_his=stock['pos_num'],
                volume_long_today=0,
                open_price_long=stock['cost'],
                name=stock['name'])
        my.add_position(pos)

In [12]:
df.groupby('code').agg({'pos_num':'sum','cost':'mean','name':'first','currency':'first'}).reset_index().apply(lambda x:add_position(x),axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object

In [14]:
my.positions.keys()

dict_keys(['stock_cn.000333', 'stock_cn.000961', 'stock_cn.002027', 'stock_cn.002353', 'stock_cn.160631', 'stock_cn.502020', 'stock_cn.600036', 'stock_cn.600516', 'stock_cn.600985', 'stock_cn.601166', 'stock_cn.601225', 'stock_cn.601318', 'stock_cn.601939'])

In [32]:
day='2021-06-25'
for code in code_cn_list:
    price=qa.QA_fetch_get_stock_day('tdx',code,day,day)
    price.apply(lambda x:my.on_price_change(code,x.close,datetime=x.date),axis=1)
# for code in code_hk_list:
#     price=qa.QA_fetch_get_hkstock_day('tdx',code,day,day) if len(code)==5 else qa.QA_fetch_get_stock_day('tdx',code,day,day)
#     price.apply(lambda x:my.on_price_change(code,x.close*0.8244,datetime=x.date),axis=1)

In [48]:
my.balance

2841682.302

In [51]:
my.ask_withdraw(1000000)

In [55]:
my.money

2496987.6763333334

In [ ]:
my.

In [47]:
my.settle()

settle


In [49]:
my.float_profit

344694.62566666666

In [36]:
my.close_profit

0

In [41]:
my.balance-my.pre_balance

344694.6256666668

In [40]:
my.position_profit

344694.62566666666

In [50]:
df1[~(df1['exchange_id']=='hk')]['balance'].sum()

1841682.302

In [17]:
df=pd.DataFrame(my.position_msg).T
df1=df[['code','name','exchange_id','volume_long_his','pos_long_his','open_price_long','last_price','position_profit','position_cost_long']].assign(balance=df['last_price']*df['pos_long_his'])
df1



,code,name,exchange_id,volume_long_his,pos_long_his,open_price_long,last_price,position_profit,position_cost_long,balance
stock_cn.000333,000333,美的集团,szse,374,374,51.69,73.17,8033.52,19332.1,27365.6
stock_cn.000961,000961,中南建设,szse,14800,14800,7.15,6,-17020,105820,88800
stock_cn.002027,002027,分众传媒,szse,20200,20200,7.392,9.15,35511.6,149318,184830
stock_cn.002353,002353,杰瑞股份,szse,300,300,32.265,45.26,3898.5,9679.5,13578
stock_cn.160631,160631,银行指基,sse,2447,2447,1,1.046,112.562,2447,2559.56
stock_cn.502020,502020,上证50增强LOF,sse,720,720,1.373,1.28,-66.96,988.56,921.6
stock_cn.600036,600036,招商银行,sse,3500,3500,42.5427,56.4,48500.7,148899,197400
stock_cn.600516,600516,方大炭素,sse,1043,1043,12.721,7.84,-5090.88,13268,8177.12
stock_cn.600985,600985,淮北矿业,sse,16024,16024,11.105,13.56,39338.9,177947,217285
stock_cn.601166,601166,兴业银行,sse,39700,39700,16.337,20.72,174005,648579,822584
